In [39]:
from scrape_stats_function import scrape_player_stats
import time
import pandas as pd
'''
    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
        if 'stats_standard' in comment:
            comment_soup = BeautifulSoup(comment, 'html.parser')
            table = comment_soup.find('table', id='stats_standard')
'''

"\n    for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):\n        if 'stats_standard' in comment:\n            comment_soup = BeautifulSoup(comment, 'html.parser')\n            table = comment_soup.find('table', id='stats_standard')\n"

In [ ]:
years = ["2024-2025","2023-2024","2022-2023","2021-2022","2020-2021","2019-2020","2018-2019","2017-2018","2016-2017","2015-2016"]
leagueinfo = [
    {
        "name": "Bundesliga",
        "url": "https://fbref.com/en/comps/20/{year}/stats/{year}-Bundesliga-Stats"
    },
    {
        "name": "Premier League",
        "url": "https://fbref.com/en/comps/9/{year}/stats/{year}-Premier-League-Stats"
    },
    {
        "name": "La Liga",
        "url": "https://fbref.com/en/comps/12/{year}/stats/{year}-La-Liga-Stats"
    },
    {
        "name": "Serie A",
        "url": "https://fbref.com/en/comps/11/{year}/stats/{year}-Serie-A-Stats"
    },
    {
        "name": "Ligue 1",
        "url": "https://fbref.com/en/comps/13/{year}/stats/{year}-Ligue-1-Stats"
    },
    {
        "name": "Eredivisie",
        "url": "https://fbref.com/en/comps/23/{year}/stats/{year}-Eredivisie-Stats"
    },
    {
        "name": "Primeira Liga",
        "url": "https://fbref.com/en/comps/32/{year}/stats/{year}-Primeira-Liga-Stats"
    },
    {
        "name": "Championship",
        "url": "https://fbref.com/en/comps/10/{year}/stats/{year}-EFL-Championship-Stats"
    },
    {
        "name": "Scottish Premiership",
        "url": "https://fbref.com/en/comps/40/{year}/stats/{year}-Scottish-Premiership-Stats"
    }
 
]

leagues = []
for year in years:
    for league in leagueinfo:
        leagues.append({
            "name": f"{league['name']} {year}",
            "url": league["url"].format(year=year),
        })

league_dfs = {}
combined_list = []
for league in leagues:
    df = scrape_player_stats(league["name"], league["url"], table_id='stats_standard')
    if df is not None:
        league_dfs[league["name"]] = df
        combined_list.append(df)
    else:
        print(f"Failed for {league['name']} at {league['url']}")
    time.sleep(5)  

200 https://fbref.com/en/comps/20/2024-2025/stats/2024-2025-Bundesliga-Stats
Found stats_standard in comment for Bundesliga 2024-2025
200 https://fbref.com/en/comps/9/2024-2025/stats/2024-2025-Premier-League-Stats
Found stats_standard in comment for Premier League 2024-2025
200 https://fbref.com/en/comps/12/2024-2025/stats/2024-2025-La-Liga-Stats
Found stats_standard in comment for La Liga 2024-2025
200 https://fbref.com/en/comps/11/2024-2025/stats/2024-2025-Serie-A-Stats
Found stats_standard in comment for Serie A 2024-2025
200 https://fbref.com/en/comps/13/2024-2025/stats/2024-2025-Ligue-1-Stats
Found stats_standard in comment for Ligue 1 2024-2025
200 https://fbref.com/en/comps/23/2024-2025/stats/2024-2025-Eredivisie-Stats
Found stats_standard in comment for Eredivisie 2024-2025
200 https://fbref.com/en/comps/32/2024-2025/stats/2024-2025-Primeira-Liga-Stats
Found stats_standard in comment for Primeira Liga 2024-2025
200 https://fbref.com/en/comps/10/2024-2025/stats/2024-2025-EFL-Cha

KeyboardInterrupt: 

In [63]:
df = pd.concat(combined_list,ignore_index=True)


df_dropped_players = df[~df['Player'].str.contains("Player", case=False, na=False)]

df_dropped_players.loc[df_dropped_players['90s'].idxmax()]


Rk                              85
Player              Sirlord Conteh
Nation                      de GER
Pos                          FW,MF
Squad                   Heidenheim
Age                             28
Born                          1996
MP                              29
Starts                           4
Min                            892
90s                            9.9
Gls                              1
Ast                              1
G+A                              2
G-PK                             1
PK                               0
PKatt                            0
CrdY                             2
CrdR                             0
xG                             2.1
npxG                           2.1
xAG                            2.6
npxG+xAG                       4.7
PrgC                            29
PrgP                            19
PrgR                            75
Gls.1                         0.10
Ast.1                         0.10
G+A.1               

In [79]:
df

,Rk,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,...,G-PK.1,G+A-PK,xG.1,xAG.1,xG+xAG,npxG.1,npxG+xAG.1,Matches,League,Season
0,1,Junior Adamu,at AUT,FW,Freiburg,23,2001,25,19,1545,...,0.12,0.23,0.32,0.14,0.46,0.32,0.46,Matches,Bundesliga 2024-2025,None
1,2,Karim Adeyemi,de GER,"FW,MF",Dortmund,22,2002,25,17,1433,...,0.44,0.82,0.34,0.38,0.72,0.34,0.72,Matches,Bundesliga 2024-2025,None
2,3,Amine Adli,ma MAR,"MF,FW",Leverkusen,24,2000,20,6,766,...,0.23,0.23,0.32,0.08,0.40,0.32,0.40,Matches,Bundesliga 2024-2025,None
3,4,Oladapo Afolayan,eng ENG,"FW,MF",St. Pauli,26,1998,32,17,1639,...,0.16,0.22,0.17,0.21,0.38,0.17,0.38,Matches,Bundesliga 2024-2025,None
4,5,Felix Agu,ng NGA,DF,Werder Bremen,24,1999,22,21,1751,...,0.15,0.15,0.07,0.09,0.17,0.07,0.17,Matches,Bundesliga 2024-2025,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4919,761,Jerry Yates,eng ENG,FW,Derby County,27,1996,42,40,3419,...,0.24,0.34,0.30,0.05,0.36,0.28,0.34,Matches,Championship 2024-2025,None
4920,762,Kusini Yengi,au AUS,"FW,MF",Portsmouth,25,1999,14,4,538,...,0.00,0.00,0.09,0.17,0.26,0.09,0.26,Matches,Championship 2024-2025,None
4921,763,Óscar Zambrano,ec ECU,MF,Hull City,20,2004,8,5,390,...,0.23,0.46,0.12,0.07,0.19,0.12,0.19,Matches,Championship 2024-2025,None
4922,764,Anass Zaroury,ma MAR,"FW,MF",Burnley,23,2000,1,0,25,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches,Championship 2024-2025,None
